## GAN 모델 학습
---
이론 참고: https://ssung-22.tistory.com/manage/newpost/57?type=post&returnURL=https%3A%2F%2Fssung-22.tistory.com%2F57

#### 패키지 Import

In [45]:
import tensorflow as tf
# tensorflow2 버전에서 1버전 코드 사용하기 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf.reset_default_graph()

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

# 데이터 불러오기 
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

import os
currentPath = os.getcwd()
print(currentPath)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
D:\포트폴리오\4학년\딥러닝


In [46]:
# parameter
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10  # 1~9 까지 총 10개 숫자 


In [47]:
X = tf.placeholder(tf.float32, [None, n_input])

# 생성자가 label을 갖고 있다면 더 좋은 성능을 가질 수 있지 않을까?
Y = tf.placeholder(tf.float32, [None, n_class])  
Z = tf.placeholder(tf.float32, [None, n_noise])

In [48]:
print("X 차원: {}".format(X.shape))
print("Y 차원: {}".format(Y.shape))
print("Z 차원: {}".format(Z.shape))

X 차원: (?, 784)
Y 차원: (?, 10)
Z 차원: (?, 128)


#### 생성자, 판별자, 노이즈 함수 생성 

In [49]:
# 가짜 이미지 생성기 
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)  # 입력 데이터에 label을 추가해서 높은 성능을 기대하자. 
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
    return output

#분류 생성기 
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()   # 변수를 재사용하며 학습 
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation=None)
    return output

# 노이즈 생성
def get_noise(batch_size, n_noise):  # batch size 만큼 균등 분폴르 따르는 노이즈 생성 
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

#### 함수 적용

In [50]:
G = generator(Z, Y)  # 그럴듯한 가짜 이미지 생성 
D_real = discriminator(X, Y)   # 진짜 이미지 판별
D_gene = discriminator(G, Y, True)  # 가짜 이미지 판별 

In [51]:
# tf.ones_like = D_real과 같은 차원의 행렬 데이터를 만들어냄. 
# tf.zeros_like = D_real과 같은 차원의 행렬 데이털르 만드는데 데이터 내 값은 모두 0
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))

loss_D = loss_D_real + loss_D_gene
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [52]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

# loss를 구하는 부분은 NN이 독립적으로 다른 NN이므로 구분 
train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [56]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})

    print('Epoch:', '%04d' % epoch, 'D loss:  {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))

    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size], Z: noise})
        
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        print('Optimization Completed!')

Epoch: 0000 D loss:  0.002247 G loss: 7.996
Optimization Completed!
Epoch: 0001 D loss:  0.002537 G loss: 8.09
Epoch: 0002 D loss:  0.01187 G loss: 9.199
Epoch: 0003 D loss:  0.001854 G loss: 8.647
Epoch: 0004 D loss:  0.0004791 G loss: 10.9
Epoch: 0005 D loss:  0.0002185 G loss: 9.222
Epoch: 0006 D loss:  0.003709 G loss: 10.89
Epoch: 0007 D loss:  0.0001053 G loss: 11.31
Epoch: 0008 D loss:  0.0005215 G loss: 10.7
Epoch: 0009 D loss:  8.387e-05 G loss: 11.75
Optimization Completed!
Epoch: 0010 D loss:  8.225e-05 G loss: 12.3
Epoch: 0011 D loss:  8.451e-06 G loss: 13.21
Epoch: 0012 D loss:  2.651e-05 G loss: 13.05
Epoch: 0013 D loss:  5.008e-08 G loss: 19.89
Epoch: 0014 D loss:  2.125e-06 G loss: 14.41
Epoch: 0015 D loss:  8.173e-06 G loss: 14.26
Epoch: 0016 D loss:  7.355e-06 G loss: 12.68
Epoch: 0017 D loss:  0.0002659 G loss: 15.85
Epoch: 0018 D loss:  3.146e-06 G loss: 28.28
Epoch: 0019 D loss:  1.105e-06 G loss: 15.54
Optimization Completed!
Epoch: 0020 D loss:  1.913e-05 G loss:

<Figure size 432x288 with 0 Axes>